In [1]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import datetime

In [2]:
def prettify(elem):
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [18]:
def map_number_to_day(day):
    if day == 0:
        return 'MONDAY'
    elif day == 1:
        return 'TUESDAY'
    elif day == 2:
        return 'WEDNESDAY'
    elif day == 3:
        return 'THURSDAY'
    elif day == 4:
        return 'FRIDAY'
    elif day == 5:
        return 'SATURDAY'
    else:
        return 'SUNDAY'

In [4]:
# input example
data = [[('Friday', '02:00:00-10:00:00'),
          ('Monday', '02:00:00-10:00:00'),
          ('Thursday', '02:00:00-10:00:00'),
          ('Tuesday', '02:00:00-10:00:00'),
          ('Wednesday', '02:00:00-10:00:00'),
          'Sachbearbeiter_#0'],
         [('Friday', '02:00:00-10:00:00'),
          ('Monday', '02:00:00-10:00:00'),
          ('Thursday', '02:00:00-10:00:00'),
          ('Tuesday', '02:00:00-10:00:00'),
          ('Wednesday', '02:00:00-10:00:00'),
          'Manager_#0']]
new_data = [
    {
        'role': 'Role 1',
        'resource': 'ID4167',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    },
    {
        'role': 'Role 1',
        'resource': 'ID4168',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    },
    {
        'role': 'Role 1',
        'resource': 'ID4169',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    },
    {
        'role': 'Role 2',
        'resource': 'ID4955',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    },
    {
        'role': 'Role 3',
        'resource': 'ID4873',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    },
    {
        'role': 'Role 4',
        'resource': 'ID4291',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    }
]

In [5]:
# TODO
# Reduce new dat to include quantity or resources
new_data = [
    {
        'role': 'Role 1',
        'quantity': '3',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    },
    {
        'role': 'Role 2',
        'quantity': '1',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    },
    {
        'role': 'Role 3',
        'quantity': '1',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    },
    {
        'role': 'Role 4',
        'quantity': '1',
        'schedule': {
            'work_days': [1, 1, 1, 1, 1, 1, 1],
            'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
            'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)
        }
    }
]

In [6]:
# Role and schedules extraction
roles = []
new_roles = []
schedules = []
new_schedules = []
new_quantities = []
for log in data:
    roles.append(log[-1])
    schedules.append(log[:-1])
for log in new_data:
    new_roles.append(log['role'])
    new_schedules.append(log['schedule'])
    new_quantities.append(log['quantity'])

In [11]:
new_schedules

[{'work_days': [1, 1, 1, 1, 1, 1, 1],
  'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
  'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)},
 {'work_days': [1, 1, 1, 1, 1, 1, 1],
  'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
  'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)},
 {'work_days': [1, 1, 1, 1, 1, 1, 1],
  'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
  'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)},
 {'work_days': [1, 1, 1, 1, 1, 1, 1],
  'start_hour': datetime.datetime(1900, 1, 1, 9, 0),
  'end_hour': datetime.datetime(1900, 1, 1, 23, 59, 59)}]

In [7]:
# namespace, will be update to tec (ITESM) namesapce
ns = {"bsim": "http://bsim.hpi.uni-potsdam.de/scylla/simModel"}

ET.register_namespace('bsim', ns["bsim"])
root = ET.Element(ET.QName(ns["bsim"], "globalConfiguration"))

resourceData =  ET.Element(ET.QName(ns["bsim"], "resourceData"))
timeTables = ET.Element(ET.QName(ns['bsim'], "timeTables"))
root.append(resourceData)
root.append(timeTables)

# For each role
for index,role in enumerate(roles):
    dynamicResource =  ET.Element(ET.QName(ns["bsim"], "dynamicResource"))
    dynamicResource.set("id", role)
    dynamicResource.set("defaultTimeUnit", "HOURS")
    tt_id = role + '_schedule'
    dynamicResource.set("defaultTimetableId", tt_id)
    # default quantity will be set to 5 until we get an algorithm to retrive the amount of concurrency on resources
    dynamicResource.set("defaultQuantity", "5")
    # Pending: Cost
    dynamicResource.set("defaultCost",'0.0')
    resourceData.append(dynamicResource)
    current_schedule = schedules[index]
    timeTable = ET.Element(ET.QName(ns['bsim'], "timeTable"))
    timeTables.append(timeTable)
    timeTable.set('id', tt_id)
    for day in current_schedule:
        timeTableItem = ET.Element(ET.QName(ns['bsim'], "timeTableItem"))
        timeTableItem.set('from',day[0].upper())
        timeTableItem.set('to',day[0].upper())
        hours = day[1].split('-')
        timeTableItem.set('beginTime', hours[0][:6])
        timeTableItem.set('endTime', hours[1][:6])
        timeTable.append(timeTableItem)

In [8]:
file = open('scylla_config_bpm.xml','w')
file.write(prettify(root))
file.close()

In [21]:
# namespace, will be update to tec (ITESM) namesapce
ns = {"bsim": "http://bsim.hpi.uni-potsdam.de/scylla/simModel"}

ET.register_namespace('bsim', ns["bsim"])
root = ET.Element(ET.QName(ns["bsim"], "globalConfiguration"))
root.set("targetNamespace","http://www.hpi.de")
root.set("id","Production_global")

resourceData =  ET.Element(ET.QName(ns["bsim"], "resourceData"))
timeTables = ET.Element(ET.QName(ns['bsim'], "timeTables"))
root.append(resourceData)
root.append(timeTables)

# For each role
for index,role in enumerate(new_roles):
    dynamicResource =  ET.Element(ET.QName(ns["bsim"], "dynamicResource"))
    dynamicResource.set("id", role)
    dynamicResource.set("defaultTimeUnit", "HOURS")
    tt_id = role + '_schedule'
    dynamicResource.set("defaultTimetableId", tt_id)
    dynamicResource.set("defaultQuantity", new_quantities[index])
    dynamicResource.set("defaultCost",'0.0')
    resourceData.append(dynamicResource)
    current_schedule = new_schedules[index]
    timeTable = ET.Element(ET.QName(ns['bsim'], "timetable"))
    timeTables.append(timeTable)
    timeTable.set('id', tt_id)
    # build timetableItems
    for i in range(7):
        if current_schedule['work_days'][i] == 1:
            timeTableItem = ET.Element(ET.QName(ns['bsim'], "timetableItem"))
            # Day
            current_day = map_number_to_day(i)
            timeTableItem.set('from', current_day)
            timeTableItem.set('to', current_day)
            # Time
            timeTableItem.set('beginTime', current_schedule['start_hour'].strftime("%H:%M"))
            timeTableItem.set('endTime', current_schedule['end_hour'].strftime("%H:%M"))
            timeTable.append(timeTableItem)

In [22]:
file = open('scylla_config_simod.xml','w')
file.write(prettify(root))
file.close()